In [12]:
import sys
import csv
import json
import requests
import requests_cache
import numpy as np
import pandas as pd
from yahoo_finance import Share
from pprint import pprint 

In [142]:
# this is a function to get the most recent n days recorded from a given company code
def myget(length, code):
    comp = Share(code)
    try:
        c = comp.get_historical('2000-11-03', '2001-01-28')
        report = []
        for i in range(0,length):
            dat = {'Symbol':c[i]['Symbol'],c[i]['Date']:float(c[i]['High'])} 
            report.append(dat)
        df = pd.DataFrame(report)
        df = df.replace(np.nan, 0).groupby('Symbol').sum()
    except YQLResponseMalformedError():
        df = None
    return df

In [119]:
# this is a function to get the most recent n days recorded from a given company code
def myget(length, code):
    comp = Share(code)
    c = comp.get_historical('2014-11-03', '2015-01-28')
    report = []
    for i in range(0,length):
        dat = {'Symbol':c[i]['Symbol'],c[i]['Date']:float(c[i]['High'])} 
        report.append(dat)
        df = pd.DataFrame(report)
        df = df.replace(np.nan, 0).groupby('Symbol').sum()
    return df

In [102]:
# yhoo= myget(10, 'TWOU')
# yhoo
tr = Share('TWOU')
tr.get_historical('2014-11-03', '2015-01-28')

[{'Adj_Close': '17.77',
  'Close': '17.77',
  'Date': '2015-01-28',
  'High': '18.08',
  'Low': '17.23',
  'Open': '18.08',
  'Symbol': 'TWOU',
  'Volume': '176400'},
 {'Adj_Close': '17.969999',
  'Close': '17.969999',
  'Date': '2015-01-27',
  'High': '18.50',
  'Low': '17.91',
  'Open': '18.00',
  'Symbol': 'TWOU',
  'Volume': '551100'},
 {'Adj_Close': '18.27',
  'Close': '18.27',
  'Date': '2015-01-26',
  'High': '18.35',
  'Low': '17.98',
  'Open': '18.07',
  'Symbol': 'TWOU',
  'Volume': '195600'},
 {'Adj_Close': '18.200001',
  'Close': '18.200001',
  'Date': '2015-01-23',
  'High': '18.33',
  'Low': '17.790001',
  'Open': '18.139999',
  'Symbol': 'TWOU',
  'Volume': '99700'},
 {'Adj_Close': '18.110001',
  'Close': '18.110001',
  'Date': '2015-01-22',
  'High': '18.290001',
  'Low': '17.785',
  'Open': '18.049999',
  'Symbol': 'TWOU',
  'Volume': '129900'},
 {'Adj_Close': '18.01',
  'Close': '18.01',
  'Date': '2015-01-21',
  'High': '18.25',
  'Low': '17.549999',
  'Open': '17.99

In [87]:
csv = pd.read_csv('./companylist.csv')
#set(csv['Sector']) # We want to keep "Finance, Health Care, Technology,Energy"
newcsv = csv[csv["Sector"].isin(["Finance", "Energy","Health Care","Technology"])]

In [89]:
final = newcsv.reset_index()
del final["index"]

In [113]:
mylist = [final.loc[i,'Symbol'] for i in range(len(final))]

In [115]:
mylist

['PIH',
 'FCCY',
 'SRCE',
 'VNET',
 'TWOU',
 'JOBS',
 'ABEO',
 'ABEOW',
 'ABIL',
 'ABMD',
 'AXAS',
 'ACIU',
 'ACIA',
 'ACHC',
 'ACAD',
 'ACST',
 'XLRN',
 'ANCX',
 'ARAY',
 'ACRX',
 'ACET',
 'AKAO',
 'ACHN',
 'ACIW',
 'ACRS',
 'ACNB',
 'ACOR',
 'ATVI',
 'ACTA',
 'ACXM',
 'ADMS',
 'ADMP',
 'ADAP',
 'ADUS',
 'IOTS',
 'ADMA',
 'ADBE',
 'ADRO',
 'AAAP',
 'AMD',
 'ADXS',
 'ADXSW',
 'ADVM',
 'AGLE',
 'AERI',
 'AEZS',
 'GNMX',
 'AFMD',
 'AGEN',
 'AGRX',
 'AGYS',
 'AGIO',
 'AGFS',
 'AGFSW',
 'AIMT',
 'AIRG',
 'AMCN',
 'AKTX',
 'AKBA',
 'AKER',
 'AKRX',
 'ALRM',
 'AMRI',
 'ALBO',
 'ADHD',
 'ALDR',
 'ALDX',
 'ALXN',
 'ALGN',
 'ALIM',
 'ALKS',
 'ABTX',
 'AIQ',
 'AHGP',
 'ARLP',
 'AHPI',
 'ALQA',
 'ALLT',
 'MDRX',
 'AFAM',
 'ALNY',
 'AOSL',
 'GOOG',
 'GOOGL',
 'ATEC',
 'AMAG',
 'AMRN',
 'AYA',
 'AMBC',
 'AMBCW',
 'AMBA',
 'DOX',
 'AMDA',
 'AMED',
 'ATAX',
 'AETI',
 'AMNB',
 'ANAT',
 'AMRB',
 'AMSWA',
 'ABCB',
 'AMSF',
 'ASRV',
 'ASRVP',
 'ATLO',
 'AMGN',
 'FOLD',
 'AMKR',
 'AMPH',
 'ASYS',
 'AFSI',

In [135]:
test1 = myget(10,mylist[1])
test1
test2 = None
test1.append(test2)

,2015-01-14,2015-01-15,2015-01-16,2015-01-20,2015-01-21,2015-01-22,2015-01-23,2015-01-26,2015-01-27,2015-01-28
Symbol,,,,,,,,,,
FCCY,11.029997,11.059996,11.109997,11.009995,11.059996,11.05,11.170004,11.490002,11.020002,11.299994


In [143]:
df=[]
del df # thi clears it so that you are not appending to previous iterations of the dataframe
for l in mylist1:
    get = myget(10,l)
    try:
        df = df.append(get) # appends if dataframe already exists
    except NameError:
        df = pd.DataFrame(get) # initialized dataframe if does not exist yet
df

NameError: global name 'YQLResponseMalformedError' is not defined

In [2]:
csv = pd.read_csv('./companylist.csv')
csv

,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9
0,PIH,"1347 Property Insurance Holdings, Inc.",7.4,4.409561e+07,n/a,2014,Finance,Property-Casualty Insurers,http://www.nasdaq.com/symbol/pih,NaN
1,FLWS,"1-800 FLOWERS.COM, Inc.",10.2,6.688044e+08,n/a,1999,Consumer Services,Other Specialty Stores,http://www.nasdaq.com/symbol/flws,NaN
2,FCCY,1st Constitution Bancorp (NJ),19.05,1.522817e+08,n/a,n/a,Finance,Savings Institutions,http://www.nasdaq.com/symbol/fccy,NaN
3,SRCE,1st Source Corporation,47.09,1.219987e+09,n/a,n/a,Finance,Major Banks,http://www.nasdaq.com/symbol/srce,NaN
4,VNET,"21Vianet Group, Inc.",7.34,4.238847e+08,57749958,2011,Technology,"Computer Software: Programming, Data Processing",http://www.nasdaq.com/symbol/vnet,NaN
5,TWOU,"2U, Inc.",36.43,1.717734e+09,n/a,2014,Technology,Computer Software: Prepackaged Software,http://www.nasdaq.com/symbol/twou,NaN
6,JOBS,"51job, Inc.",35.97,9.637576e+08,26793371,2004,Technology,Diversified Commercial Services,http://www.nasdaq.com/symbol/jobs,NaN
7,CAFD,8point3 Energy Partners LP,13.39,1.058783e+09,n/a,2015,Public Utilities,Electric Utilities: Central,http://www.nasdaq.com/symbol/cafd,NaN
8,EGHT,8x8 Inc,15.65,1.423099e+09,n/a,n/a,Public Utilities,Telecommunications Equipment,http://www.nasdaq.com/symbol/eght,NaN
9,AVHI,"A V Homes, Inc.",16.55,3.735354e+08,n/a,n/a,Capital Goods,Homebuilding,http://www.nasdaq.com/symbol/avhi,NaN


In [ ]:
sym = csv 

In [6]:
def lookup(jurisdiction, number):
    url = 'http://api.opencorporates.com/companies/' + jurisdiction + '/' + number
    response = requests.get(url)
    if response.status_code == 200:
        data = json.loads(response.text)
        company = data['results']['company']
        return {
            'name': company['name'],
            'type': company['company_type'],
            'incorporated': company['incorporation_date'],
            'address': company['registered_address_in_full']
        }
    else:
        print('Error: ' + str(response.status_code))

In [7]:
print(lookup(sys.argv[1], sys.argv[2]))


Error: 404
None


In [9]:
sys.argv

['/Users/audreychu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py',
 '-f',
 '/Users/audreychu/Library/Jupyter/runtime/kernel-2a43a2c9-8519-458f-990a-ad9d9651c8a6.json']